In [27]:
epochs = 20
n_train_items = 6000
train_item_per_client = 600
rounds = 20
C = 0.1
total_client = 100
n_workers = int(total_client * C)

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import random

In [29]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
# simulation functions
def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]


workers = connect_to_workers(n_workers=n_workers)


In [30]:
class Arguments():
    def __init__(self):
        self.batch_size = 10
        self.test_batch_size = 60
        self.epochs = epochs
        self.rounds = rounds
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 2
        self.save_model = False
        self.n_train_items = n_train_items
        self.train_item_per_client = train_item_per_client

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [32]:
# federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ]))
#     .federate(workers), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
#     batch_size=args.batch_size, shuffle=True, **kwargs)

# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args.test_batch_size, shuffle=True, **kwargs)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

def create_dataset_for_client(train_loader):
    train_dataloader_client = []
    tmp = []
    for count, batch_data in enumerate(train_loader):
        if (count * args.batch_size) % (args.train_item_per_client) == 0:
            #Data in tmp is equal to the train_item_per_client
            train_dataloader_client.append(tmp)
            tmp = []
        else :
            #Keep appending the data into tmp
            tmp.append(batch_data)
    return train_dataloader_client


train_dataloader_client = create_dataset_for_client(train_loader)




In [33]:
# less_train_dataloader[0][0].shape
len(train_dataloader_client)

100

In [29]:
# from PIL import Image
# import numpy 
# #mnist_dataset.__getitem__(2)[1]
# a = (mnist_dataset.__getitem__(0)[0]).numpy()
# a.dtype = 'uint8'
# print(a)
# Image.fromarray(a[0], mode= 'P')

In [1]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, 1)
        self.conv2 = nn.Conv2d(32, 64, 5, 1)
        self.fc1 = nn.Linear(3136, 512)
        self.fc2 = nn.Linear(512, 10)
        self.same_padding = nn.ReflectionPad2d(2)
#         self.batch32 = nn.BatchNorm2d(32)
#         self.batch64 = nn.BatchNorm2d(64)

    def forward(self, x):
        x = self.same_padding(x)
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
#         x = self.batch32(x)
        x = self.same_padding(x)
        x = F.relu(self.conv2(x))
#         x = self.batch64(x)
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 3136)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

NameError: name 'nn' is not defined

In [41]:
def model_init(workers, Net):
    model_list = list()
    for worker in workers:
        model_list.append(Net)
    return model_list
def opt_init(model_list):
    opt_list = list()
    for model  in model_list:
        opt_list.append(optim.SGD(model.parameters(), lr=args.lr))
    return opt_list
def random_sample(train_dataloader):
    choice_list = sorted(random.sample(range(100), 10))
    count = 0
    tmp = []
    for i, data in enumerate(train_dataloader):
        if  i == choice_list[count]:
            tmp.append(data)
            if count == 9:
                pass
            else:
                count += 1
    return tmp

In [42]:
def train(args, device, train_loader, opt_list, workers):
    global model_list
    ## start training and record the model into model_list
    less_train_dataloader = random_sample(train_loader)
    for epoch in range(args.epochs):
        for batch_idx, (data, target) in enumerate(less_train_dataloader): # <-- now it is a distributed dataset
            model_on_worker = model_list[batch_idx%len(workers)]
            model_on_worker.train()
            model_on_worker.send(workers[batch_idx%len(workers)]) # <-- NEW: send the model to the right location

            data_on_worker = data.send(workers[batch_idx%len(workers)])
            target_on_worker = target.send(workers[batch_idx%len(workers)])

            data_on_worker, target_on_worker = data_on_worker.to(device), target_on_worker.to(device)

            opt_list[batch_idx%len(workers)].zero_grad()

            output = model_on_worker(data_on_worker)
            loss = F.cross_entropy(output, target_on_worker)
            loss.backward()

            opt_list[batch_idx%len(workers)].step()
            model_on_worker.get() # <-- NEW: get the model back

            model_list[batch_idx%len(workers)] = model_on_worker #When len(dataloader) is longer than the len(worker) send and get must be modified
            #model_list here is full of the model which has trained on the workers, there are all different now.

        if epoch % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, n_train_items, n_train_items ,
                100. * epoch / args.epochs, loss.item()))


    ##Aggregation time
    new_model = []
    tmp_model = Net().to(device)
    with torch.no_grad():
        for p in model_list[0].parameters():
            new_model.append(0)
            
        for m in model_list:
            for par_idx, par in enumerate(m.parameters()):
                #average the model_list
                new_model[par_idx] = new_model[par_idx]+par.data
                # we get new model in list format and need to set_ to model
        for worker in range(len(workers)):
            for par_idx in range(len(new_model)):
                list(model_list[worker].parameters())[par_idx].set_(new_model[par_idx]/len(workers))
        #init model with new_model
        


In [43]:
def test(args, model, device, test_loader, r):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)*(args.test_batch_size)
    accuracy = 100. * correct / (len(test_loader)*args.test_batch_size)
    #Since the test loader here is a list, we can get the len by * it with batch.size
    
    
    writer.add_scalar('Accuracy', accuracy,r)
    writer.add_scalar('Loss', test_loss, r)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader)* (args.test_batch_size),
        accuracy))

In [34]:
logdir = '/root/notebooks/tensorflow/logs/pysyft_shuffle6000_crossE'
writer = SummaryWriter(logdir)


In [35]:
#test
# model_test = Net()
# output_test = model_test(less_train_dataloader[0][0])
# loss_test = F.cross_entropy(output_test, less_train_dataloader[0][1])
# print(loss_test)

In [39]:
%%time
#optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment


model_list = []
model_list = model_init(workers, Net().to(device))
opt_list = opt_init(model_list)
# not finish in train, finish latter
pars = [list(model.parameters()) for model in model_list]

for r in range(1, args.rounds + 1):
    train(args, device, less_train_dataloader, opt_list, workers)
    print("After training")
    test(args, model_list[0], device, test_loader, r)


if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 0 [6000/6000 (0%)]	Loss: 0.122490
Train Epoch: 2 [6000/6000 (10%)]	Loss: 0.092055
Train Epoch: 4 [6000/6000 (20%)]	Loss: 0.056337
Train Epoch: 6 [6000/6000 (30%)]	Loss: 0.022451
Train Epoch: 8 [6000/6000 (40%)]	Loss: 0.016670
Train Epoch: 10 [6000/6000 (50%)]	Loss: 0.012220
Train Epoch: 12 [6000/6000 (60%)]	Loss: 0.011827
Train Epoch: 14 [6000/6000 (70%)]	Loss: 0.006277
Train Epoch: 16 [6000/6000 (80%)]	Loss: 0.004746
Train Epoch: 18 [6000/6000 (90%)]	Loss: 0.003852
After training

Test set: Average loss: 0.0847, Accuracy: 9775/10020 (98%)

Train Epoch: 0 [6000/6000 (0%)]	Loss: 0.003286
Train Epoch: 2 [6000/6000 (10%)]	Loss: 0.002789
Train Epoch: 4 [6000/6000 (20%)]	Loss: 0.002313
Train Epoch: 6 [6000/6000 (30%)]	Loss: 0.001893
Train Epoch: 8 [6000/6000 (40%)]	Loss: 0.001610
Train Epoch: 10 [6000/6000 (50%)]	Loss: 0.001363
Train Epoch: 12 [6000/6000 (60%)]	Loss: 0.001206
Train Epoch: 14 [6000/6000 (70%)]	Loss: 0.001054
Train Epoch: 16 [6000/6000 (80%)]	Loss: 0.000932
Train